In [1]:
import sys
sys.path.append('u2net')
from u2net.prediction_class import Predict
import cv2
import numpy as np
import os

**TEST:**
сделали бинаризацию, сделали блюр по старому, результат в final, результат блюра без бинаризации в output

**TEST 1:**
сначала делаем бинаризацию, потом делаем блюр по новому

**TEST 2:**
сначала делаем бинаризацию, потом блюр по старому, потом блюр по новому

**TEST 3:**
сначала делаем бинаризацию, потом блюр по новому, потом блюр по старому

TEstы 2 и 3 не нужны, потому что бинаризация отрезает волосы

Надо понять, на сколько вообще нужна бинаризация, потому что ее надо либо уменьшать, либо убирать совсем

И после нее надо ставить блюр по новому

**TEST 1:**

In [20]:
image = '/home/ubuntu/removebg/test_1/test.jpg'
model_name = 'u2net'

In [21]:
img = cv2.imread(image, cv2.IMREAD_ANYCOLOR)

In [22]:
# prediction = Predict(model_name)
# prediction.predict(image)

In [23]:
mask_path = '/home/ubuntu/removebg/test_1/test.png'

In [24]:
matting = cv2.imread(mask_path, cv2.IMREAD_ANYCOLOR)

In [25]:
##### бинаризация 

th, im_th = cv2.threshold(matting, 150, 255, cv2.THRESH_BINARY)
cv2.imwrite(mask_path[:-4]+'_binarized.png', im_th)
binarized = cv2.imread(mask_path[:-4]+'_binarized.png')

kernel = np.ones((5,5), np.uint8)
binarized_smoothed = cv2.erode(binarized,kernel,iterations = 1)

cv2.imwrite(mask_path[:-4]+'_binarized_eroded.png', binarized_smoothed) 
binarized_eroded = cv2.imread(mask_path[:-4]+'_binarized_eroded.png', cv2.IMREAD_GRAYSCALE)

In [28]:
mask_ = binarized_eroded #cv2.imread(mask_path)
blurred_mask = cv2.GaussianBlur(mask_, (21, 21), 0)
mask_of_mask = np.zeros(mask_.shape, np.uint8)

# gray = cv2.cvtColor(mask_, cv2.COLOR_BGR2GRAY)
# cv2.imwrite(mask_path[:-4]+'_gray.png', gray)

thresh = cv2.threshold(mask_of_mask, 60, 255, cv2.THRESH_BINARY)[1]
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

res = cv2.drawContours(mask_of_mask, contours, -1, (255,255,255),5)
cv2.imwrite(mask_path[:-4]+'_res.png', res)

output = np.where(mask_of_mask==np.array([255, 255, 255]), blurred_mask, mask_)
cv2.imwrite(mask_path[:-4]+'_output.png', output)

result_mask = cv2.imread(mask_path[:-4]+'_output.png', cv2.IMREAD_GRAYSCALE)
##### удаление фона 

result = cv2.bitwise_and(img, img, mask=result_mask)
result[result_mask==0] = [255,255,255]

cv2.imwrite(mask_path[:-4]+'_final.png', result)

/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


True

In [28]:
img.shape

(640, 960, 3)

In [29]:
output.shape

(640, 960, 3)

In [12]:
##### блюр или блюр через erode 


#binarized_smoothed = cv2.blur(binarized, (5,5),0)
kernel = np.ones((5,5), np.uint8)
binarized_smoothed = cv2.erode(binarized,kernel,iterations = 1)

cv2.imwrite(mask_path[:-4]+'_binarized_smoothed.png', binarized_smoothed) 
result_mask = cv2.imread(mask_path[:-4]+'_binarized_smoothed.png', cv2.IMREAD_GRAYSCALE)

In [14]:
binarized_smoothed.shape

(640, 960, 3)

In [13]:
result_mask.shape

(640, 960)

In [13]:
##### удаление фона 


result = cv2.bitwise_and(img, img, mask=result_mask)
result[result_mask==0] = [255,255,255]

cv2.imwrite(mask_path[:-4]+'_final.png', result)

In [38]:
# os.remove(mask_path[:-4]+'_binarized.png')
# os.remove(mask_path[:-4]+'_binarized_smoothed.png')

In [15]:
mask_ = cv2.imread(mask_path)
blurred_mask = cv2.GaussianBlur(mask_, (21, 21), 0)
mask_of_mask = np.zeros(mask_.shape, np.uint8)

In [24]:
gray = cv2.cvtColor(mask_, cv2.COLOR_BGR2GRAY)
cv2.imwrite(mask_path[:-4]+'_gray.png', gray)

True

In [25]:
thresh = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)[1]

In [26]:
contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [27]:
res = cv2.drawContours(mask_of_mask, contours, -1, (255,255,255),5)
cv2.imwrite(mask_path[:-4]+'_res.png', res)

True

In [29]:
output = np.where(mask_of_mask==np.array([255, 255, 255]), blurred_mask, mask_)
cv2.imwrite(mask_path[:-4]+'_output.png', output)

True